In [ ]:
from progressivis_nb_widgets.nbwidgets import *

In [ ]:
import pandas as pd
from progressivis.core import Scheduler, Every
from progressivis.table import Table
from progressivis.vis import MCScatterPlot
from progressivis.io import ParquetLoader
#from progressivis.datasets import get_dataset
from progressivis.table.constant import Constant
import asyncio as aio
import pyarrow as pa
import threading
import os

def _quiet(x): pass

def _filter(bat):
    df = bat.to_pandas()
    pklon = df['pickup_longitude']
    pklat = df['pickup_latitude']
    dolon = df['dropoff_longitude']
    dolat = df['dropoff_latitude']
    res = df[(pklon > -74.08) & (pklon < -73.5) & (pklat > 40.55) & (pklat < 41.00) &
                  (dolon > -74.08) & (dolon < -73.5) & (dolat > 40.55) & (dolat < 41.00)]
    return pa.RecordBatch.from_pandas(res)

try:
    s = scheduler
except NameError:
    s = Scheduler()

PREFIX = "../nyc-taxi/bk_500k_yellow_tripdata_2015"

URLS = [f"{PREFIX}-{month:0>2}.parquet" for month in range(1,7)]


FILENAMES = pd.DataFrame({'filename': URLS})
CST = Constant(Table('filenames', data=FILENAMES), scheduler=s)
CSV = ParquetLoader(index_col=False, skipinitialspace=True,
                columns=['pickup_longitude', 'pickup_latitude',
                             'dropoff_longitude', 'dropoff_latitude'],
                filter_=_filter, scheduler=s) # TODO: reimplement filter in read_csv.py

CSV.input.filenames = CST.output[0]
PR = Every(scheduler=s, proc=_quiet)
PR.input[0] = CSV.output.result


MULTICLASS = MCScatterPlot(scheduler=s, classes=[
    ('pickup', 'pickup_longitude', 'pickup_latitude'),
    ('dropoff', 'dropoff_longitude', 'dropoff_latitude')], approximate=True)
MULTICLASS.create_dependent_modules(CSV)

# Create the dashboard object
psboard = PsBoard(s, order='desc')
# Visualisations require registration :
# 1) When widget provides the link_module() method do :
psboard.register_visualisation(Scatterplot(disable=['init_centroids']), MULTICLASS)
# 2) When widget is generic or for overloading the link_module() method do:
# psboard.register_visualisation(FooWidget(), bar_module, glue=glue_func)
# NB: glue_func(widget, module)-> [coroutine] must be provided
# Start application :

s.task_start();

#### Click  _&#x1f441; mc_scatter_plot_1_ below to display the scatterplot:

In [ ]:
display(psboard)